In [ ]:
import numpy as np
import pandas as pd

### Data Cleansing (Part 1)

In [ ]:
df = pd.read_csv('./Dataset2020-2021.csv')

### Clean vitek_id

In [ ]:
df.loc[df[df.vitek_id == 'Strep'].index, 'vitek_id'] = 'GP'

In [ ]:
for i, v in df[(df.vitek_id != 'GN') & (df.vitek_id != 'GP')].filename.items():
    vitek = v.split('_')
    if vitek[4] == 'GP' or vitek[4] == 'GN':
        df.loc[i,'vitek_id'] = vitek[4]
    elif vitek[3] == 'GP' or vitek[3] == 'GN':
        df.loc[i,'vitek_id'] = vitek[3]
    else:
        print(v)

### Clean Sex

In [ ]:
df.sex = df.sex.str.lower()
df.loc[df[df.sex == 'unk'].index, 'sex'] = 'unknown'
df.loc[df[df.sex == 'cat'].index, 'sex'] = 'unknown'

### Clean species

In [ ]:
df.species = df.species.str.lower()
df.species = df.species.str.strip()
df.loc[df[(df.species != 'cat') & (df.species != 'dog')].index, 'species'] = 'other'

### จัด format ของ report_issued_date

In [ ]:
month_dict = {'มกราคม':'1', 'มกราาคม':'1', 'กุมภาพันธ์':'2', 'มีนาคม':'3', 'เมษายน':'4', 'พฤษภาคม':'5', 'มิถุนายน':'6', 'กรกฎาคม':'7', 'สิงหาคม':'8', 'กันยายน':"9", 'ตุลาคม':"10", 'พฤศจิกายน':"11", 'ธันวาคม':"12"}
for i, v in df.report_issued_date.items():
    # print(f"Index : {i}, Value : {v}")
    date = str(v).split()
    if date[0] != 'nan' and len(date) == 4:
        # print(df.loc[i, 'report_issued_date'])
        df.loc[i, 'report_issued_date'] = date[0] + '/' + month_dict[date[1]] + '/' + str(int(date[-1].replace('พ.ศ.', ''))-543)
    else:
        print(v)

### จัด format ของ date_of_submission

In [ ]:
month_eng_dict = {'Jan' : '1', 'Feb' : '2', 'Mar' : '3', 'March' : '3', 'Apr' : '4', 'May' : '5', 'June' : '6', 'Jun' : '6', 'July' : '7', 'Jul' : '7', 'Aug' : '8', 'Sep':'9', 'Oct':'10', 'Nov' : '11', 'Dec':'12'}
for i, v in df.date_of_submission.items():
    date = str(v).split()
    if len(date) == 3:
        df.loc[i, 'date_of_submission'] = f"{date[0]}/{month_eng_dict[date[1]]}/{date[2]}"
    elif len(date) == 2:
         df.loc[i, 'date_of_submission'] = f"1/{month_eng_dict[date[0]]}/{date[1]}"
    else:
        print(v)

### จัด format ของ sample_collection_date

In [ ]:
month_eng_dict = {'Jan' : '1', 'Feb' : '2', 'Mar' : '3', 'March' : '3', 'Apr' : '4', 'May' : '5', 'June' : '6', 'Jun' : '6', 'July' : '7', 'Jul' : '7', 'Aug' : '8', 'Sep':'9', 'Oct':'10', 'Nov' : '11', 'Dec':'12'}
for i, v in df.sample_collection_date.items():
    date = str(v).split()
    if len(date) == 3:
        df.loc[i, 'sample_collection_date'] = f"{date[0]}/{month_eng_dict[date[1]]}/{date[2]}"
    elif len(date) == 2:
         df.loc[i, 'sample_collection_date'] = f"1/{month_eng_dict[date[0]]}/{date[1]}"
    else:
        print(v)

### แปลงเป็น Datetime

In [ ]:
df.report_issued_date = pd.to_datetime(df.report_issued_date, format='%d/%m/%Y')
df.sample_collection_date = pd.to_datetime(df.sample_collection_date, format='%d/%m/%Y')
df.date_of_submission = pd.to_datetime(df.date_of_submission, format='%d/%m/%Y')

### Clean bact_species

In [ ]:
df.bact_species = df.bact_species.str.lower()
df.loc[df[df.bact_species.str.contains('no bacterial growth')].index, 'bact_species'] = 'no bacterial growth'
df.loc[df[df.bact_species.str.contains('yeast')].index, 'bact_species'] = 'yeast'

In [ ]:
for i, v in df.loc[df[df.bact_species.str.contains('low discrimination')].index, 'bact_species'].items():
    bactName = v.replace('low discrimination', "").replace('(', '').replace('between', '').replace('among', '').strip()
    if "กับ" in bactName:
        bactName = bactName[:bactName.index('กับ')].strip()
        df.loc[i, 'bact_species'] = bactName
    if "/" in bactName:
        bactName = bactName[:bactName.index('/')].strip()
        df.loc[i, 'bact_species'] = bactName
    if " and " in bactName:
        bactName = bactName[:bactName.index(' and ')].strip()
        df.loc[i, 'bact_species'] = bactName
    if " or " in bactName:
        bactName = bactName[:bactName.index(' or ')].strip()
        df.loc[i, 'bact_species'] = bactName
    if "," in bactName:
        bactName = bactName[:bactName.index(',')].strip()
        df.loc[i, 'bact_species'] = bactName
    if bactName == '':
        df.loc[i, 'bact_species'] = 'unknown'
    if bactName == '6.3x106 cfu':
        df.loc[i, 'bact_species'] = 'unknown'

In [ ]:
df.loc[df[df.bact_species.str.contains('non or low reactive biopattern')].index, 'bact_species'] = 'unknown'
df.loc[df[df.bact_species.str.contains('unidentified')].index, 'bact_species'] = 'unknown'

In [ ]:
for i, v in df.bact_species.items():
    bactName = v
    if "(" in bactName:
        bactName = v[:v.index('(')].strip()
        df.loc[i, 'bact_species'] = bactName
    if "/" in bactName:
        bactName = v[:v.index('/')].strip()
        df.loc[i, 'bact_species'] = bactName
    if " or " in bactName:
        bactName = v[:v.index(' or ')].strip()
        df.loc[i, 'bact_species'] = bactName
    if "หรือ" in bactName:
        bactName = v[:v.index('หรือ')].strip()
        df.loc[i, 'bact_species'] = bactName
    if "กับ" in bactName:
        bactName = v[:v.index('กับ')].strip()
        df.loc[i, 'bact_species'] = bactName
    if "," in bactName:
        bactName = v[:v.index(',')].strip()
        df.loc[i, 'bact_species'] = bactName
    if " by " in bactName:
        bactName = v[:v.index(' by ')].strip()
        df.loc[i, 'bact_species'] = bactName
    if "." in bactName:
        bactName = v.replace('.', '').strip()
        df.loc[i, 'bact_species'] = bactName

### เพิ่มคอลัมน์ bacteria_genus

In [ ]:
df['bact_genus'] = np.nan

In [ ]:
df.loc[df[df.bact_species.str.contains('staphylococci')].index, 'bact_genus'] = 'staphylococci'
df.loc[df[df.bact_species.str.contains('staphylococcus')].index, 'bact_genus'] = 'staphylococcus'

In [ ]:
ng_list = ['no bacterial and fungal growth', 'no bacterial growth', 'no fungal growth', 'no microbial growth']
for i, v in df.bact_genus.items():
    if str(v) == 'nan':
        if df.loc[i, 'bact_species'] not in ng_list :
            df.loc[i, 'bact_genus'] = df.loc[i, 'bact_species'].split()[0]
        else:
            df.loc[i, 'bact_genus'] = 'no growth'

In [ ]:
df.loc[df[df.bact_genus == 'gram'].index, 'bact_genus'] = 'unknown'

### Clean submitted_sample

In [ ]:
df['submitted_sample'] = df['submitted_sample'].str.lower()
df['submitted_sample'] = df['submitted_sample'].str.strip()
df['submitted_sample'].fillna('unknown', inplace=True)

In [ ]:
# ตัดวงเล็บออก
for i, v in df['submitted_sample'].items():
    sample = str(v)
    if "(" in sample:
        sample = v[:v.index('(')].strip()
        df.loc[i, 'submitted_sample'] = sample

### Clean collect_method

In [ ]:
df['collect_method'].fillna('unknown', inplace=True)
df['collect_method'] = df['collect_method'].str.lower()
df['collect_method'] = df['collect_method'].str.strip()

In [ ]:
df.loc[df[df['collect_method'] == '-'].index, 'collect_method'] = 'unknown'
df.loc[df[df['collect_method'] == 'unk'].index, 'collect_method'] = 'unknown'

### Export CSV

In [ ]:
df.to_csv('Dataset2020-2021_clean.csv', index=False)